In this notebook, you'll work on clustering hyperspectral image data, using dimensionality reduction to help guide your work. 

It accompanies Chapter 7 of the book.

Copyright: Viviana Acquaviva (2023).

Additions and modifications by Julieta Gruszko (2025).

License: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/)

### Hyperspectral image data

Hyperspectral images are data that contain both spatial (position x, y) and spectral (wavelength $\lambda$) data. For each pixel in an image, information about the spectral emission on that pixel is corrected, as shown in this image:
 ![A set of stones is scanned with a Specim LWIR-C imager in the thermal infrared range from 7.7 μm to 12.4 μm. The quartz and feldspar spectra are clearly recognizable.](./HSI_LWIR_stones.png)

 Image Credit: Holma, H., (May 2011), Thermische Hyperspektralbildgebung im langwelligen Infrarot, Photonik



Hyperspectral imaging is useful, for example, in aerial surveys, to identify different types of terrain based on their spectral signature and location. Of course, usual color photos already contain spectral information, since they tell you the average emission in the red, green, and blue (RGB) parts of the spectrum. But the frequency bands used in that case are very wide; what we call hyperspectral images contain many more frequency bands that are much more narrow. 

Due to this difference, the way we treat them in this clustering example is different: instead of treating each image as an instance, and each pixel of the image (or maybe the red, green, and blue versions of each pixel) as a feature, as we did in our galaxy classification task, we'll treat each pixel as an instance, and the flux at each frequency as a feature. We'll only be looking at a single image, and we'll seek to cluster/classify the pixels of the image. 

In this homework, we'll work with a "classic" data set: aerial images of the city of Pavia, Italy, collected by the European Reflective Optics System Imaging Spectrometer (ROSIS-3) satellite. We'll use the "Pavia Center" data set, which consists of one composite image of 1,096 x 715 pixels, with a 1.3m spatial resolution. Each pixel is measured in 102 spectral bands between 430 and 834nm. 

Our goal is to figure out what type of land-cover each pixel of the image represents. Some of the pixels are labeled, which will let us check our work, but our goal is to do this task using unsupervised learning tools. So you know what we're working with, the labels and frequencies in the data set are:

| Label | Class | Membership |
|:---------|:--------:|---------:|
| 0 | Unlabeled | 635488 |
| 1 | Water | 65971 |
| 2 | Trees | 7598 |
| 3 | Asphalt | 3090 |
| 4 | Self-blocking Bricks | 2685 |
| 5 | Bitumen | 6584 |
| 6 | Tiles | 9248 |
| 7 | Shadows | 7287 |
| 8 | Meadows| 42826 |
| 9 | Bare Soil | 2863 |




In [1]:
import numpy as np

In [2]:
from matplotlib import pyplot as plt
from matplotlib import cm
import matplotlib

font = {'size'   : 16}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
matplotlib.rcParams.update({'figure.autolayout': False})
matplotlib.rcParams['figure.dpi'] = 300

In [3]:
import scipy.io
from sklearn import metrics
from sklearn.metrics import accuracy_score, adjusted_rand_score
from sklearn.metrics import pairwise
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA

### Import Pavia centre data set 

Here we read the data (an image of the center of Pavia, in Italy) and the ground truth labels that assign a classification to each of the pixels. 

Unfortunately, the uncompressed Pavia center data set is a bit too big to be stored using Github's normal file distribution system. Instead, the data available on the Canvas site. They're linked in the "Problem Sets" module. You'll need to download them and point the commands below to the correct directory on your computer. 

In [ ]:
mat = scipy.io.loadmat('/path/to/Pavia.mat')

In [5]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'pavia'])

In [6]:
mat['pavia'].shape

(1096, 715, 102)

In [7]:
data = mat['pavia']

### Question: 
Describe the structure and contents of the file that you just loaded, and explain how you'll use it in your clustering algorithm, assuming we'll be following the approach described above for hyperspectral images. Include a description of everything in the file, not just what we've called the $\texttt{data}$ object, but make sure you also describe how the $\texttt{data}$ object is structured and what it contains. 

How many instances and how many features will we be using?

Using the same approach as above, load the $\texttt{Pavia\_gt.mat}$ file, check the keys contained in it, and save the ground truth labels to a new data structure $\texttt{labels}$. Check the shape of the labels and the unique values found in it. Then, using the table above, make a dictionary that connects each numerical value to the correct descriptive text label. The keys should be the integer labels, and the values should be the text labels. If you don't have any experience with Python dictionaries, you can find some documentation here: https://docs.python.org/3/tutorial/datastructures.html#dictionaries

In [9]:
# code to load the file and check the contain keys
labels = # store the ground truth labels here
# then check the shape and unique values


In [11]:
gt_names = # make a dictionary to map integer labels to the descriptive text labels

### Question: 
Describe the structure and contents of the file that you just loaded. Include a description of everything in the file, not just what we've called the $\texttt{labels}$ object, but make sure you also describe how the $\texttt{labels}$ object is structured and what it contains. 


### Visualize map and ground truth

We follow the existing literature to reproduce a false color composite RGB (bands 10, 27, 46):

In [13]:
np.max(data[:,:,10]+data[:,:,27]+data[:,:,46])

In [14]:
plt.figure(figsize = (12,20))
plt.imshow(data[:,:,10]+data[:,:,27]+data[:,:,46], interpolation='nearest', cmap='gist_gray', vmax = 10000)
plt.tight_layout()
#plt.savefig('PaviaCentreMap.pdf', dpi = 300)
plt.show()

As you may be able to tell, this is a composite image made up of two aerial images stuck together. We can also show the ground truth labels:

In [15]:
plt.figure(figsize = (20,12))
plt.imshow(labels, interpolation='nearest', cmap='viridis')
plt.colorbar()
plt.show()

Let's take a look at label distribution:

In [16]:
for i in range(len(np.unique(labels.flatten()))):
    print(i, len(np.where(labels.flatten() == i)[0]), gt_names[i])

In [17]:
gt_names.values()

Note that because the number 0 is not associated to a class, we will get rid of pixels with label = 0.

Next, we'll create the feature matrix, using a mask to drop any unlabeled pixels.

In [18]:
data.shape

In [19]:
mask = (labels != 0)

In [20]:
data[mask]

Get rid of pixels with label = 0 for the clustering, and mask the labels the same way

In [21]:
maskdata = data[mask]

In [22]:
maskdata.shape # ~148,000 instances (pixels), 102 attributes

In [23]:
masklabels = labels[mask]

#### Next, scale the data before clustering.

We'll want the scaling we apply to work well for k-means and PCA, since that's what we'll be doing to start with. What properties should it have? Explain your answer, and then scale the data using an appropriate sk-learn scaler. 

In [24]:
maskdata_scaled = # scale your data 

Run clustering using k-means and the known number of labels. 

This isn't stricly unsupervised learning since we're using the known number of classes; later we'll change this.

Carefully consider your initialization strategy and number of initializations! 

Get the cluster assignments for all the data points using $\texttt{fit\_predict}$.

In [26]:
# code to run k-means clustering on your scaled data and get the cluster assignments for all points.
clusters = # save the assigned clusters here

By default the labels used run from 0 to 8, we'll adjust them to run 1-9, like our truth labels.

In [28]:
clusters = clusters + 1 # rename labels to be between 1 and 9

In [29]:
np.unique(clusters)

We can look at the distribution of objects in clusters to get a general idea of whether the clustering was successful. I've started you off with a plot of the ground truth labels, just add a histogram of your cluster labels on top.

In [ ]:
plt.figure(figsize = (12,6))
plt.hist(masklabels.flatten(), alpha = 0.5, color = 'm', label = 'True', bins = 9, range = (0.5,9.5))
# plot an overlapping histogram of your cluster results
plt.title('Clustering results by $k$-means, $k$ = 9', fontsize=16)
plt.legend(fontsize = 16);
plt.xlabel('Class or cluster ID', fontsize = 16);
plt.ylabel('Number of samples', fontsize = 16);
plt.xticks(np.arange(1,10));
#plt.savefig('kmeans9prediction.pdf', dpi = 300)

Remember, the indexing is not significant (clusters are assigned at random)!

### Question:
What can you guess about your algorithm's success in clustering the most common and second most common categories, from these statistics?

### Adjusted Rand Index

The Adjusted Rand Index gives an idea of how much two distributions are similar, and can be used to compare two clustering schemes. It varies between 0 and 1, with 0 being "equivalent to random" and 1 being perfectly equivalent. We get a value of ~0.77, not bad, although as we learned with any measure of similarity, there is no immediate interpretability.

In [31]:
ari_kmeans = adjusted_rand_score(masklabels.flatten(), clusters)

In [32]:
ari_kmeans

While there is no "absolute interpretation" of the ARI value, a value of 0.77 indicates that the original distribution and the one found by k-means are in reasonable agreement.

### Find centroids from ground truth

A useful exercise for this problem is to calculate "accuracy" by assigning labels to predicted clusters (note that in most clustering problems, ground truth labels are not available - this is a special case!) 

For this, we need to map each cluster found by k-means to one of the ground truth labels. This is non-trivial, but we will attempt to do it by identifying each "predicted" centroid with the closest ground truth centroid.

We start by calculating the ground truth centroids:

In [33]:
gt_centroids = []

for i in range(1, len(np.unique(masklabels.flatten()))+1): #remember that labels go from 1 to 9
    indices = (np.where(masklabels.flatten() == i)[0])
    gt_centroids.append(np.mean(maskdata_scaled[indices,:], axis = 0))

Next, we calculate the matrix of distances, in feature space, between ground-truth-based (GT) centroids and clustering-based (CB) centroids:

In [34]:
C = pairwise.euclidean_distances(gt_centroids, kmeans.cluster_centers_)

In [35]:
# Plot the matrix

#From here: https://stackoverflow.com/questions/32503308/plot-distance-matrix-for-a-1d-array
    
plt.figure(figsize = (20,20))

plt.matshow(C,cmap="Blues",fignum=1)

n = int(np.sqrt(C.size))

ax = plt.gca()

# Set the plot labels
xlabels = ["CB%d" % i for i in range(n+1)]
ylabels = ["GT%d" % i for i in range(n+1)]
#print(xlabels)
ax.set_xticklabels(xlabels)
ax.set_yticklabels(ylabels)

#Add text to the plot showing the values at that point
for i in range(n):
    for j in range(n):
        plt.text(j,i, np.round(C[i,j],2), horizontalalignment='center', verticalalignment='center')

plt.show()

We can now map each clustering-based centroid to the closest GT centroid:

In [36]:
#Find closest GT centroid for each CB centroid; assign corresponding label 

for i in range(kmeans.n_clusters):
    print(i+1, np.argmin(C[:,i])+1, np.round(C[np.argmin(C[:,i]),i],4)) 

Note that this does not guarantee a 1:1 mapping! Some ground truth labels may not be getting assigned to any k-means cluster, and multiple k-means clusters may be associated with the same ground truth label.

Additionally, we can see that the distances to closest centroids vary quite a bit; we will use visualization techniques in a moment to see if we can interpret this behavior as less separated/identifiable clusters.

### Question:
What ground truth labels don't seem to be assigned to any k-means cluster, if any?

Create a dictionary to re-assign the predicted cluster membership, so we can calculate label-based metrics, such as accuracy. The mapping should go from current predicted label (as the key) to GT label (as the value).

In [ ]:
d = # your remapping dictionary

One final complication is that we also have the background pixels (with label 0), which will be part of the final predicted image (but not participate in the accuracy calculation, of course).

In [38]:
indices = np.where(labels.flatten() != 0)[0] #these are the non-background pixels

In [39]:
len(indices)

In [40]:
predlabels = np.zeros(len(labels.flatten())) #start with all-background labels of 0

In [41]:
predlabels[indices] = clusters #assign predicted cluster membership where label is not 0

In [42]:
predlabels_remap = np.copy(predlabels)

for k, v in d.items(): predlabels_remap[predlabels == k] = v #re-map labels according to the new labeling scheme we created

We can check manually that the re-labeling worked as expected. Make sure the outputs here match what you expect.

In [43]:
predlabels[200:300]

In [44]:
predlabels_remap[200:300]

We are finally ready to calculate the accuracy score. Calculate it using the original labels and remapped labels, using sk-learn's $\texttt{accuracy\_score}$.

In [ ]:
# code to get accuracy score

Is this too good to be true? Certainly so, because we should remember to exclude background pixels (which are, in fact, the majority)! Try again, using only the indices corresponding to non-background pixels:

In [ ]:
# code to get accuracy score of non-background pixels

### Question:
What is the accuracy score for the labeled (non-background) pixels, using 9 k-means clusters?

Overall, the score you see shouldn't be too bad - because we know one class comprises almost half the object, we can also invoke a full confusion matrix or consider a class-by-class accuracy. Using sk-learn's $\texttt{confusion\_matrix}$ option from $\texttt{metrics}$, make a confusion matrix showing percentages.

In [ ]:
np.set_printoptions(precision=2, suppress=True) #Suppress scientific notation

# make the confusion matrix and display it

Finally, we can display the GT and predicted images (after making sure that the color scale is the same - here it's not trivial, because the predicted labels only vary between 1 and 8):

In [66]:
f, axarr = plt.subplots(1,2,figsize=(15,10.5));

# Add legend mapping colors to classes
norm = matplotlib.colors.Normalize(vmin=0, vmax=9)
for i in range(1, kmeans.n_clusters+1):
    rgba_color = cm.viridis(norm(i)) 
    plt.scatter(0,0, s = 20, c = np.atleast_2d(rgba_color), label = gt_names[i], marker='s')
plt.legend(loc=[1.1,0.535], markerscale=3, fontsize=18)

# Plot ground truth vs predictions
axarr[0].imshow(labels, interpolation='None', cmap='viridis', vmin = 0, vmax = 9);
axarr[0].set_title('True', fontsize = 16)
pl = axarr[1].imshow(predlabels_remap.reshape(1096, 715), interpolation='None', cmap='viridis', vmin = 0, vmax = 9);
axarr[1].set_title('Predicted ($k$-means, $k$ = 9)',fontsize = 16);
plt.tight_layout()
#plt.savefig('GTvskmeans9.pdf', dpi = 300)

### Questions:
- What differences do you observe? You may have to look carefully. Identify at least two differences.

- What categories is our clustering algorithm capturing well? Which ones is it capturing poorly?

We can visualize "incorrect" assignments below by creating a 0/1 mask where 0 are correctly assigned pixels, and 1 and incorrectly assigned pixels:

In [49]:
colormask = np.array([labels.flatten() == predlabels_remap]).astype(int) 

In [50]:
f, axarr = plt.subplots(1,2,figsize=(15,15));
axarr[0].imshow(labels, interpolation='None', cmap='viridis');
axarr[0].set_title('True', fontsize = 16)
pl = axarr[1].imshow(colormask.reshape(1096, 715), interpolation='None', cmap = 'gray')#;
axarr[1].set_title('Incorrect assignments', fontsize = 16);

### Next step: Visualization with PCAs

We can take a look at the hyperspectral signature of some random pixels.

In [51]:
np.random.seed(10)
for i in np.random.randint(0,42000,10):
    plt.plot(range(102),maskdata_scaled[i])

When no "spiky" features are present, we can expect a decent performance from linear PCAs. 

Apply a linear PCA to your scaled, masked data. 

Then, make a plot of the cumulative explained variance as a function of the number of components, as we did in Studio 11. As we did there, you don't need to include all of the possible PCA components in the plot, but make sure you include enough to see where the improvement flattens out.

In [ ]:
transformer = # our PCA model 
# code to apply linear PCA and make a plot of cumulative explained variance. Make sure you label your axes!

### Questions: 
- How many components are needed to encode 99% of the variance? How much of the variance is encoded in just the first two components?

- Based on your answer, should we expect a 2-D visualization of the data to be useful?

We'll plot data on the 2D PCA projections, just to get an idea of separability. I've given you the code to set up the color scale,  show the ground truth labels, and label the axes. You'll need to add the code to plot the transformed data, color-coding it by the ground truth label. Look back at the Studio 11 code if you need a reminder on how to transform the data. 

Hint: use $\texttt{c = masklabels}$ in your plotting command to set the point color using the labels. 

In [ ]:
plt.figure(figsize = (16,10))

# Add legend mapping colors to classes
norm = matplotlib.colors.Normalize(vmin=0, vmax=9)
rgba_colors = [cm.viridis(norm(i)) for i in range(1, kmeans.n_clusters+1)]
   
# add your code to make the scatter plot here



plt.text(transformer.transform(np.array(gt_centroids))[0,0],\
            transformer.transform(np.array(gt_centroids))[0,1]+0.9,str(gt_names[1]), fontsize = 16) #Add "water" 
                                                                                                    #label separately
    
for i in range(2, kmeans.n_clusters+1):
    plt.text(transformer.transform(np.array(gt_centroids))[i-1,0]+0.1,\
             transformer.transform(np.array(gt_centroids))[i-1,1]+0.3,str(gt_names[i]), fontsize = 16)


plt.xlabel('PCA 1', fontsize =16)
plt.ylabel('PCA 2', fontsize =16)      
plt.xlim(-10,25)
plt.ylim(-12,10);

#plt.savefig('PCAmap.pdf', dpi = 300)

### Question:
Using the PCA plot, interpret the results of the k-means clustering. Can you explain why the clustering performed well on the groups that it did, and poorly on the groups that it did?

We can also understand more the mapping between clusters are classes by plotting the ground truth + predicted centroids. Again, I've given you most of the setup, but you'll need to add scatter plots of the transformed ground truth and k-means centroids.

In [ ]:
plt.figure(figsize = (16,10))

# Add legend mapping colors to classes
norm = matplotlib.colors.Normalize(vmin=0, vmax=9)
rgba_colors = [cm.viridis(norm(i)) for i in range(1, kmeans.n_clusters+1)]
   

# scatter plot of transformed ground truth centroids

# scatter plot of transformed k-means centroids


for i in range(1, kmeans.n_clusters+1):
    plt.text(transformer.transform(np.array(gt_centroids))[i-1,0]+0.1,\
             transformer.transform(np.array(gt_centroids))[i-1,1]+0.3,str(gt_names[i]), fontsize = 16)
    plt.text(transformer.transform(np.array(kmeans.cluster_centers_))[i-1,0]-0.1,\
             transformer.transform(np.array(kmeans.cluster_centers_))[i-1,1]-0.5,str(i), fontsize = 16)
plt.xlabel('PCA 1', fontsize =16)
plt.ylabel('PCA 2', fontsize =16)      

plt.legend(fontsize =16);

#plt.savefig('PCAcentroids.pdf', dpi = 300)

### Questions:
- Of the 9 clusters you asked k-means to create, which seem to correspond to ground truth clusters, based on the PCA analysis? 
- Are there any k-means clusters you'd suggest merging to better align with the ground truth labels? If so, describe which you'd merge, and what ground truth label they correspond to. This type of post-processing "cluster merging" is pretty common when you're using unsupervised methods for science. 

One last question that we could ask is: if we had treated this problem as a classic unsupervised problem, where the number of clusters is not known a priori, what would have we found?

We can try using a few tools that we have learned about. The simplest choice for this case is probably the Elbow method; the silhouette score can also work, but because it requires calculating pairwise distances among all objects, it would be prohibitive to calculate it on the full data set (we could solve this with random sampling).

### Question:
Based on the PCA analysis, should we expect the elbow method to perform reasonably well on this data? Explain your reasoning.

Make an elbow method plot for this data, using k-means. You'll need to decide for yourself what values you want to sample for numbers of clusters. Go back to Studio 10 if you need a reminder on how to do this. 

In [ ]:
# your code to get the intertias for varying numbers of clusters

In [ ]:
# your code to make an elbow curve plot

Based on your elbow curve, pick a preferred number of clusters and re-run the k-means clustering. Then check the histogram of class distributions, the adjusted Rand score, and the accuracy on non-background pixels, using the same strategy we used above to map the cluster labels to appropriate ground truth labels.

In [ ]:
# code to re-run k-means
# code to make the histogram comparison to ground truth labels
# code to determine labels using distances and re-assign labels with a dictionary
# code to determine accuracy on non-background pixels


Finally, we can visualize the map of ground truth labels vs predictions (you may need to change variable names to get this code to run on your results!):

In [76]:
f, axarr = plt.subplots(1,2,figsize=(15,15));

# Add legend mapping colors to classes
norm = matplotlib.colors.Normalize(vmin=0, vmax=9)
for i in range(1, kmeans5.n_clusters+1):
    rgba_color = cm.viridis(norm(i)) 
    plt.scatter(0,0, s = 20, c = np.atleast_2d(rgba_color), label = gt_names[i], marker='s')
plt.legend(loc=[1.1,0.57], markerscale=3, fontsize=18)

# Plot ground truth vs predictions
axarr[0].imshow(labels, interpolation='None', cmap='viridis', vmin = 0, vmax = 9);
axarr[0].set_title('True', fontsize = 16)
pl = axarr[1].imshow(predlabels5_remap.reshape(1096, 715), interpolation='None', cmap='viridis', vmin = 0, vmax = 9);
axarr[1].set_title('Predicted, k-Means, k = 5',fontsize = 16);
#plt.colorbar(pl,ax=axarr[:], shrink = 0.6)

### Question:
How does the accuracy using your chosen number of clusters compare with the accuracy using 9 clusters?

### GMM 

In our final experiment, we use Gaussian Mixture Models to see if they are better at picking up the correct number of clusters and at generating a clustering scheme.

As in Studio 10, use the BIC to decide how many components to use. It's up to you to decide whether you should allow the GMM to use full covariances, or if you think we should restrict them. Warning: this can be slow, particularly if you decide to study many models with full covariances! You'll need to balance the need to get useful information with computational feasibility.

Then cluster the data with however many components your BIC indicates is optimal; check the histogram of class distributions, the adjusted Rand score, and the accuracy on non-background pixels, using the same strategy we used above to map the cluster labels to appropriate ground truth labels.



In [ ]:
# GMM code

### Questions:
- How does the GMM performance compare with both examples of the k-means performance? 
- Based on the label frequency histogram, how is GMM performing on the 2 most-frequent classes?
- Using the PCA analysis, explain why GMM performs as it does on this data, compared with k-means. 

Finally, we can visualize the map of ground truth labels vs predictions (you may need to change variable names to get this code to run on your results!). If you're interested, you can also repeat the PCA centroid analysis for the GMM, but you're not required to!

In [91]:
f, axarr = plt.subplots(1,2,figsize=(15,10.5));

# Add legend mapping colors to classes
norm = matplotlib.colors.Normalize(vmin=0, vmax=9)
for i in range(1, len(np.unique(labels))):
    rgba_color = cm.viridis(norm(i)) 
    plt.scatter(0,0, s = 20, c = np.atleast_2d(rgba_color), label = gt_names[i], marker='s')
plt.legend(loc=[1.1,0.535], markerscale=3, fontsize=18)

# Plot ground truth vs predictions
axarr[0].imshow(labels, interpolation='None', cmap='viridis', vmin = 0, vmax = 9);
axarr[0].set_title('True', fontsize = 16)
pl = axarr[1].imshow(predlabelsGMM_remap.reshape(1096, 715), interpolation='None', cmap='viridis', vmin = 0, vmax = 9);
axarr[1].set_title('Predicted (GMM, n$_{comp}$ = 8)',fontsize = 16);
plt.tight_layout()
#plt.savefig('GTvsGMM8.pdf', dpi = 300)

It's quite interesting to note that in the GMM map, there is very little "salt and pepper" noise - classes may be wrong, but they are wrong "in blocks", even without using spatial information. We can visualize this behavior by taking a look at the incorrect pixel map:

In [92]:
colormask = np.array([labels.flatten() == predlabelsGMM_remap]).astype(int) 

In [93]:
f, axarr = plt.subplots(1,2,figsize=(15,15));
axarr[0].imshow(labels, interpolation='None', cmap='viridis');
axarr[0].set_title('True', fontsize = 16)
pl = axarr[1].imshow(colormask.reshape(1096, 715), interpolation='None', cmap = 'gray')#;
axarr[1].set_title('Incorrect assignments', fontsize = 16);

References:

Data: http://www.ehu.eus/ccwintco/index.php?title=Hyperspectral_Remote_Sensing_Scenes#Pavia_University_scene

Classification: https://arxiv.org/pdf/1612.00144v2.pdf reports kNN accuracy on scaled data set of 77% on 9 classes

Ref [80] in master thesis: https://arxiv.org/pdf/1704.07961.pdf (bunch of clustering results on 6-class data set)

Master's thesis: https://www.ri.cmu.edu/wp-content/uploads/2020/05/Masters_thesis_2020-1.pdf

### Acknowledgement Statement: